In [45]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/features.csv")

Мне пришлось использовать только 250000 строк для обучения, потому что иначе колаб падал (и ноутбук тоже). На качестве моделей это отразилось незначительно.

In [27]:
features = features.loc[:250000, :]

## 3.3

In [6]:
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [71]:
train = features[['last_letters_name', 'last_letters_surname', 'last_letters_patronymic', 'len_name', 'len_surname', 'len_patronymic']]

X_train, X_valid, y_train, y_valid = train_test_split(train, features['target'], test_size=0.2, random_state=13)

In [72]:
# я взяла этот код со scikit-learn.org

numeric_features = ['len_name','len_surname','len_patronymic']
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ['last_letters_name', 'last_letters_surname', 'last_letters_patronymic']
categorical_transformer = OneHotEncoder(dtype=bool, sparse=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

## 3.4

In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# import lightgbm as lgb - я приняла решение не разбираться с lgb в этой домашке ради экономии ресурсов

In [38]:
def get_roc_auc(X_train, X_valid, y_train, y_valid, model, preprocessor):
    p = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])
    p.fit(X_train, y_train)
    y_true = p.predict(X_valid)
    score = roc_auc_score(y_valid, y_true)
    return score

In [74]:
get_roc_auc(X_train, X_valid, y_train, y_valid, LinearRegression(), preprocessor)

0.9973432629856015

In [14]:
get_roc_auc(X_train, X_valid, y_train, y_valid, LogisticRegression(), preprocessor)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9978999849142193

In [75]:
get_roc_auc(X_train, X_valid, y_train, y_valid, DecisionTreeClassifier(), preprocessor)

0.9969230216697331

In [76]:
get_roc_auc(X_train, X_valid, y_train, y_valid, RandomForestClassifier(), preprocessor)

0.9977744016546635

In [77]:
get_roc_auc(X_train, X_valid, y_train, y_valid, XGBClassifier(), preprocessor)

0.9977136395274954

Нельзя сказать что какая-то модель работает лучше, а какая-то хуже, они все показывают значения roc-auc > 0.996. Как минимум 98.8% данных (посчитала по таблице features) однозначно определяются с помощью окончания отчества (_на_ или _ич_), остальные 1.2% и вызывают небольшие колебания в качествах моделей. Еще в данных встречается неправильная разметка, и это тоже влияет на колебания.

## 4: Загрузка тестовых данных в модель

Первая половина части 4 находится в другой тетрадке

In [12]:
test = pd.read_csv("/content/drive/MyDrive/features_test.csv")

Я выбрала линейную регрессию, потому что она работала быстрее всего. 

In [13]:
p = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", LinearRegression())])
p.fit(X_train, y_train)
predictions = p.predict(test)

In [15]:
df = pd.DataFrame(predictions)
df = pd.concat([test['id'], df], axis=1)
df['target'] = df[0].apply(lambda x: float(round(x)))
df = df.drop(columns=[0])

In [17]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions.csv', index=False)

## 5.1

#### Модель для сравнения

Сначала уберем длину отчества - она у всех одинаковая, это показано выше.

In [28]:
exp1_features = features[['id', 'last_letters_surname', 'last_letters_name', 'last_letters_patronymic', 'len_name', 'len_surname', 'target']]

In [29]:
exp1_train = exp1_features[['last_letters_surname',	'len_name',	'len_surname', 'last_letters_name',	'last_letters_patronymic']]
exp1_X_train, exp1_X_valid, exp1_y_train, exp1_y_valid = train_test_split(exp1_train, exp1_features['target'], test_size=0.2, random_state=13)

In [39]:
numeric_features = ['len_name','len_surname']
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ['last_letters_name', 'last_letters_surname', 'last_letters_patronymic']
categorical_transformer = OneHotEncoder(dtype=bool, sparse=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Все улучшения я хочу сравнивать с этой моделью. Я выбрала линейную регрессию, потому что она обучается быстрее всего.


In [40]:
get_roc_auc(exp1_X_train, exp1_X_valid, exp1_y_train, exp1_y_valid, LinearRegression(), preprocessor)

0.9971469561303072

#### Добавляем фичи

Я получила этот файл в другой тетрадке

In [61]:
new_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/new_features.csv")

In [62]:
new_features

,id,target,vtoc_surname,numpe
0,165386,1.0,-1,0
1,243807,0.0,0,0
2,3970023,1.0,-2,0
3,3670414,0.0,-1,0
4,3477203,0.0,0,0
...,...,...,...,...
249996,1689621,1.0,-1,0
249997,5283149,0.0,-1,0
249998,3061177,0.0,-2,0
249999,4654739,1.0,-1,0


Первая фича - разница количества гласных к количеству согласных в фамилии. Гипотеза в том, что в мужских фамилиях согласных больше, чем гласных, в женских - наоборот.

In [ ]:
exp2_features = exp1_features
exp2_features['vtoc_surname'] = new_features['vtoc_surname']
# почему эта ячейка не запущена? я не знаю, так скопировалось из колаба :/

In [65]:
exp2_train = exp2_features[['last_letters_surname',	'len_name',	'len_surname', 'last_letters_name',	'last_letters_patronymic', 'vtoc_surname']]
exp2_X_train, exp2_X_valid, exp2_y_train, exp2_y_valid = train_test_split(exp2_train, exp2_features['target'], test_size=0.2, random_state=13)

In [66]:
numeric_features = ['len_name','len_surname', 'vtoc_surname']
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ['last_letters_name', 'last_letters_surname', 'last_letters_patronymic']
categorical_transformer = OneHotEncoder(dtype=bool, sparse=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [67]:
get_roc_auc(exp2_X_train, exp2_X_valid, exp2_y_train, exp2_y_valid, LinearRegression(), preprocessor)

0.9978025817311601

Стало работать лучше на 0.0007!

Вторая фича - количество букв п в имени. Гипотеза: в женских именах п меньше, чем в мужских.

In [68]:
exp3_features = exp1_features
exp3_features['numpe'] = new_features['numpe']

In [69]:
exp3_train = exp3_features[['last_letters_surname',	'len_name',	'len_surname', 'last_letters_name',	'last_letters_patronymic', 'numpe']]
exp3_X_train, exp3_X_valid, exp3_y_train, exp3_y_valid = train_test_split(exp3_train, exp3_features['target'], test_size=0.2, random_state=13)

In [70]:
numeric_features = ['len_name','len_surname', 'numpe']
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ['last_letters_name', 'last_letters_surname', 'last_letters_patronymic']
categorical_transformer = OneHotEncoder(dtype=bool, sparse=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

get_roc_auc(exp3_X_train, exp3_X_valid, exp3_y_train, exp3_y_valid, LinearRegression(), preprocessor)

0.9970565910908152

Стало работать хуже на 0.0001 - количество букв _п_ в именах не влияет